# Scanning the BP dataset with pysegy

This notebook demonstrates how to download the BP 1994 2-D seismic dataset, scan it using the `pysegy` utilities and then visualise the source and receiver positions as well as a few shot gathers.

In [ ]:
import sys
sys.path.append('..')  # allow importing the local pysegy package
import pysegy
import urllib.request, gzip, os, shutil
import numpy as np
import matplotlib.pyplot as plt


In [ ]:
bp_url = 'http://s3.amazonaws.com/open.source.geoscience/open_data/bpmodel94/Model94_shots.segy.gz'
local_gz = 'Model94_shots.segy.gz'
local_segy = 'Model94_shots.segy'

if not os.path.exists(local_segy):
    print('Downloading BP dataset...')
    with urllib.request.urlopen(bp_url) as resp, open(local_gz, 'wb') as f:
        shutil.copyfileobj(resp, f)
    with gzip.open(local_gz, 'rb') as gz, open(local_segy, 'wb') as out:
        shutil.copyfileobj(gz, out)


In [ ]:
def scan_file(path):
    with open(path, 'rb') as f:
        fh = pysegy.read.read_fileheader(f)
        ns = fh.bfh.ns
        dsf = fh.bfh.DataSampleFormat
        trace_size = 240 + ns * 4
        f.seek(0, os.SEEK_END)
        ntraces = (f.tell() - 3600) // trace_size
        f.seek(3600)
        shots = []
        offsets = []
        counts = []
        current = None
        count = 0
        pos = 3600
        for i in range(ntraces):
            th = pysegy.read.read_traceheader(f)
            if current is None:
                current = (th.SourceX, th.SourceY)
                offsets.append(pos)
            if (th.SourceX, th.SourceY) != current:
                shots.append(current)
                counts.append(count)
                current = (th.SourceX, th.SourceY)
                offsets.append(pos)
                count = 0
            count += 1
            f.seek(ns * 4, os.SEEK_CUR)
            pos += trace_size
        shots.append(current)
        counts.append(count)
    return fh, np.array(shots), np.array(offsets), np.array(counts)


In [ ]:
fh, shots, offsets, counts = scan_file(local_segy)
print(f'Total shots: {len(shots)}')
print('Samples per trace:', fh.bfh.ns)


In [ ]:
plt.figure(figsize=(6,5))
plt.scatter(shots[:,0], shots[:,1], s=4)
plt.xlabel('SourceX')
plt.ylabel('SourceY')
plt.title('Source Positions')
plt.axis('equal')
plt.show()


In [ ]:
def read_gather(f, offset, count, ns):
    f.seek(offset)
    headers = []
    traces = np.zeros((ns, count), dtype=np.float32)
    for i in range(count):
        th = pysegy.read.read_traceheader(f)
        headers.append(th)
        raw = f.read(ns * 4)
        traces[:, i] = [pysegy.ibm.ibm_to_ieee(raw[j:j+4]) for j in range(0, ns*4, 4)]
    return headers, traces

with open(local_segy, 'rb') as f:
    for i in range(3):
        off = int(offsets[i])
        cnt = int(counts[i])
        hdrs, data = read_gather(f, off, cnt, fh.bfh.ns)
        plt.figure(figsize=(8,4))
        plt.imshow(data[::-1], aspect='auto', cmap='gray')
        plt.title(f'Shot gather {i+1}')
        plt.xlabel('Trace')
        plt.ylabel('Sample')
        plt.show()


In [ ]:
# receiver positions for first shot
off = int(offsets[0])
count = int(counts[0])
with open(local_segy, 'rb') as f:
    hdrs, _ = read_gather(f, off, count, fh.bfh.ns)
rx = [h.GroupX for h in hdrs]
ry = [h.GroupY for h in hdrs]
plt.figure(figsize=(6,5))
plt.scatter(rx, ry, s=4)
plt.scatter(shots[0,0], shots[0,1], c='r', marker='x', label='source')
plt.xlabel('X')
plt.ylabel('Y')
plt.title('Receiver positions for first shot')
plt.legend()
plt.axis('equal')
plt.show()
